In [ ]:
from glob import glob
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)

i=0
# For every data set in the data repository...
for dataset in glob('data/car-evaluation.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    #print(input_data)
    
    for dataset_repeat in range(1, 31):
        print(i)
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        loss_list=['hinge', 'log','modified_huber','squared_hinge','perceptron']
        penalty_list=['l2','l1','elasticnet']
        alpha_list = [0.000001, 0.00001, 0.0001, 0.001, 0.01]
        learning_rate_list = ['constant','optimal','invscaling']
        warm_start_list = ['True', 'False']
        
        # For every SGDClassifier parameter combination...
        for (loss, penalty, alpha, learning_rate, warm_start) in itertools.product(loss_list, 
                                                                                   penalty_list,
                                                                                   alpha_list,
                                                                                   learning_rate_list,
                                                                                   warm_start_list):
            # Create and fit the model on the training data
            clf = SGDClassifier(loss=loss, penalty=penalty, alpha=alpha, learning_rate=learning_rate, warm_start=warm_start)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'SGDClassifier',
                              (loss, penalty, alpha, learning_rate, warm_start))].append(clf.score(testing_features,
                                                                                      testing_classes))
            
        i=i+1
print(all_model_scores)